In [1]:
from loaders.Loaders import CSVLoader
from compoundFeaturization.rdkitFingerprints import MorganFingerprint
from featureSelection.baseFeatureSelector import LowVarianceFS
from splitters.splitters import SingletaskStratifiedSplitter
from models.sklearnModels import SklearnModel
from metrics.Metrics import Metric
from metrics.metricsFunctions import r2_score, roc_auc_score, precision_score, accuracy_score, confusion_matrix, classification_report, f1_score
from parameterOptimization.HyperparameterOpt import HyperparamOpt_CV, HyperparamOpt_Valid

from sklearn.ensemble import RandomForestClassifier

In [2]:
#Load Dataset
dataset = CSVLoader(dataset_path='preprocessed_dataset_wfoodb.csv', 
                    mols_field='Smiles', 
                    labels_fields='Class', 
                    id_field='ID')#, shard_size=4000)
dataset = dataset.create_dataset()
dataset.get_shape()

Mols_shape:  23290
Features_shape:  X not defined!
Labels_shape:  (23290,)


In [3]:
#Featurization
dataset = MorganFingerprint().featurize(dataset)
dataset.get_shape()

Featurizing datapoint 0
Featurizing datapoint 1000
Featurizing datapoint 2000
Featurizing datapoint 3000
Featurizing datapoint 4000
Featurizing datapoint 5000
Featurizing datapoint 6000


RDKit ERROR: [16:15:41] Explicit valence for atom # 1 Cl, 4, is greater than permitted


error in smile: O=[Cl]=O
Featurizing datapoint 7000


RDKit ERROR: [16:15:44] Explicit valence for atom # 3 B, 4, is greater than permitted
RDKit ERROR: [16:15:44] Explicit valence for atom # 1 Cl, 9, is greater than permitted


error in smile: OB1O[B]2(O)OB(O)O[B](O)(O1)O2
error in smile: O=[Cl-](=O)(=O)=O
Featurizing datapoint 8000
Featurizing datapoint 9000
Featurizing datapoint 10000
Featurizing datapoint 11000
Featurizing datapoint 12000
Featurizing datapoint 13000
Featurizing datapoint 14000


RDKit ERROR: [16:15:58] Explicit valence for atom # 0 P, 11, is greater than permitted


error in smile: [P](OCC=C(C)C)(OCC=C(C)C)(=O)(OP(OCC=C(C)C)(OCC=C(C)C)=O)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)CC=C(C)C
Featurizing datapoint 15000
Featurizing datapoint 16000
Featurizing datapoint 17000
Featurizing datapoint 18000
Featurizing datapoint 19000
Featurizing datapoint 20000
Featurizing datapoint 21000
Featurizing datapoint 22000
Featurizing datapoint 23000
Elements with indexes:  [6257, 7708, 7709, 14244]  were removed due to the presence of NAs!
The elements in question are:  ['O=[Cl]=O' 'OB1O[B]2(O)OB(O)O[B](O)(O1)O2' 'O=[Cl-](=O)(=O)=O'
 '[P](OCC=C(C)C)(OCC=C(C)C)(=O)(OP(OCC=C(C)C)(OCC=C(C)C)=O)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)CC=C(C)C']
Mols_shape:  23286
Features_shape:  (23286, 1024)
Labels_shape:  (23286,)


In [4]:
#Feature Selection
dataset = LowVarianceFS(0.15).featureSelection(dataset)
dataset.get_shape()

Mols_shape:  23286
Features_shape:  (23286, 49)
Labels_shape:  (23286,)


In [5]:
#Data Split
splitter = SingletaskStratifiedSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset=dataset, frac_train=0.6, 
                                                                             frac_valid=0.2, frac_test=0.2)

In [6]:
#Scikit-Learn Random Forest
rf = RandomForestClassifier()
model = SklearnModel(model=rf)

In [7]:
#cross validation
model.cross_validate(dataset, Metric(roc_auc_score), folds=3)

Computing Stratified K-fold split

Split 1 :
Train Score: 
roc_auc_score: 
 0.8521023762598879
Test Score: 
roc_auc_score: 
 0.6416251769263241

Split 2 :
Train Score: 
roc_auc_score: 
 0.8530715477862341
Test Score: 
roc_auc_score: 
 0.6522147173307242

Split 3 :
Train Score: 
roc_auc_score: 
 0.8576459889137484
Test Score: 
roc_auc_score: 
 0.649837806998226


(SklearnModel(model=RandomForestClassifier()),
 0.8530715477862341,
 0.6522147173307242,
 [0.8521023762598879, 0.8530715477862341, 0.8576459889137484],
 [0.6416251769263241, 0.6522147173307242, 0.649837806998226],
 0.8542733043199568,
 0.6478925670850915)

In [8]:
# model training
model.fit(train_dataset)

RandomForestClassifier()

In [9]:
metrics = [Metric(roc_auc_score), Metric(precision_score), Metric(accuracy_score), Metric(confusion_matrix), 
           Metric(classification_report)]
print("#############################")
# evaluate the model
print('Training Dataset: ')
train_score = model.evaluate(train_dataset, metrics)
print("#############################")
print('Validation Dataset: ')
valid_score = model.evaluate(valid_dataset, metrics)
print("#############################")
print('Test Dataset: ')
test_score = model.evaluate(test_dataset, metrics)
print("#############################")

#############################
Training Dataset: 
roc_auc_score: 
 0.8599885501535931
precision_score: 
 0.9391447368421053
accuracy_score: 
 0.981672394043528
confusion_matrix: 
 [[13141    37]
 [  219   571]]
classification_report: 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99     13178
           1       0.94      0.72      0.82       790

    accuracy                           0.98     13968
   macro avg       0.96      0.86      0.90     13968
weighted avg       0.98      0.98      0.98     13968

#############################
Validation Dataset: 
roc_auc_score: 
 0.6539265606888558
precision_score: 
 0.5782312925170068
accuracy_score: 
 0.9482388316151202
confusion_matrix: 
 [[4330   62]
 [ 179   85]]
classification_report: 
               precision    recall  f1-score   support

           0       0.96      0.99      0.97      4392
           1       0.58      0.32      0.41       264

    accuracy                           0

In [10]:
#Build a model function for hyperparameter optimization
def rf_model_builder(n_estimators=10, max_features='auto', class_weight={0: 1., 1: 1.}):
    rf_model = RandomForestClassifier(n_estimators=n_estimators, max_features=max_features, 
                                      class_weight=class_weight)
    return rf_model

params_dict_rf = {"n_estimators": [10, 100],
                  "max_features": ["auto", "sqrt", "log2", None],
                  "class_weight": [{0: 1., 1: 1.}, {0: 1., 1: 5}, {0: 1., 1: 10}]
                  }


In [11]:
#Hyperparameter Optimization
optimizer = HyperparamOpt_Valid(rf_model_builder)

best_rf, best_hyperparams, all_results = optimizer.hyperparam_search(params_dict_rf, 
                                                                     train_dataset, 
                                                                     valid_dataset, 
                                                                     Metric(accuracy_score))

print('#################')
print(best_hyperparams)
print(best_rf)

#Evaluate model
best_rf.evaluate(test_dataset, metrics)

MODE:  classification
Fitting 15 random models from a space of 24 possible models.
Fitting model 1/15
hyperparameters: {'n_estimators': 10, 'max_features': 'auto', 'class_weight': {0: 1.0, 1: 1.0}}
expected str, bytes or os.PathLike object, not NoneType
accuracy_score: 
 0.9437285223367697
Model 1/15, Metric accuracy_score, Validation set 1: 0.943729
	best_validation_score so far: 0.943729
Fitting model 2/15
hyperparameters: {'n_estimators': 10, 'max_features': 'auto', 'class_weight': {0: 1.0, 1: 10}}
expected str, bytes or os.PathLike object, not NoneType
accuracy_score: 
 0.9231099656357389
Model 2/15, Metric accuracy_score, Validation set 2: 0.923110
	best_validation_score so far: 0.943729
Fitting model 3/15
hyperparameters: {'n_estimators': 10, 'max_features': 'sqrt', 'class_weight': {0: 1.0, 1: 5}}
expected str, bytes or os.PathLike object, not NoneType
accuracy_score: 
 0.938573883161512
Model 3/15, Metric accuracy_score, Validation set 3: 0.938574
	best_validation_score so far: 

{'roc_auc_score': 0.6430255344727542,
 'precision_score': 0.5374149659863946,
 'accuracy_score': 0.9460910652920962,
 'confusion_matrix': 1164.0,
 'classification_report': None}

In [12]:
#Hyperparameter Optimization with CV
optimizer = HyperparamOpt_CV(rf_model_builder)

best_rf, best_hyperparams, all_results = optimizer.hyperparam_search('sklearn',
                                                                     params_dict_rf, 
                                                                     train_dataset,  
                                                                     'accuracy', 
                                                                     cv=3,
                                                                     n_iter_search=10)

print('#################')
print(best_hyperparams)
print(best_rf)

MODEL TYPE:  sklearn
asdasdas
Fitting 10 random models from a space of 24 possible models.

 
 Best accuracy: 0.941294 using {'n_estimators': 100, 'max_features': 'log2', 'class_weight': {0: 1.0, 1: 1.0}}

 accuracy: 0.932417 (0.005238) with: {'n_estimators': 100, 'max_features': 'log2', 'class_weight': {0: 1.0, 1: 5}} 


 accuracy: 0.925759 (0.006198) with: {'n_estimators': 10, 'max_features': 'sqrt', 'class_weight': {0: 1.0, 1: 10}} 


 accuracy: 0.927262 (0.005417) with: {'n_estimators': 10, 'max_features': 'auto', 'class_weight': {0: 1.0, 1: 10}} 


 accuracy: 0.928121 (0.005010) with: {'n_estimators': 100, 'max_features': None, 'class_weight': {0: 1.0, 1: 5}} 


 accuracy: 0.931701 (0.004124) with: {'n_estimators': 10, 'max_features': 'auto', 'class_weight': {0: 1.0, 1: 5}} 


 accuracy: 0.941294 (0.002661) with: {'n_estimators': 100, 'max_features': 'log2', 'class_weight': {0: 1.0, 1: 1.0}} 


 accuracy: 0.929482 (0.005074) with: {'n_estimators': 10, 'max_features': 'log2', 'clas

In [13]:
#Evaluate model
best_rf.evaluate(test_dataset, metrics)

roc_auc_score: 
 0.6426841598710246
precision_score: 
 0.5266666666666666
accuracy_score: 
 0.945446735395189
confusion_matrix: 
 [[4323   71]
 [ 183   79]]
classification_report: 
               precision    recall  f1-score   support

           0       0.96      0.98      0.97      4394
           1       0.53      0.30      0.38       262

    accuracy                           0.95      4656
   macro avg       0.74      0.64      0.68      4656
weighted avg       0.94      0.95      0.94      4656



{'roc_auc_score': 0.6426841598710246,
 'precision_score': 0.5266666666666666,
 'accuracy_score': 0.945446735395189,
 'confusion_matrix': 1164.0,
 'classification_report': None}

In [14]:
import sklearn
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])